In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline     
## use `%matplotlib notebook` for interactive figures
# plt.style.use('ggplot')

import pandas as pd
import numpy as np
import pickle
import math
import sklearn
import glob,sys
from tqdm.auto import tqdm
from sklearn.linear_model import LinearRegression
import proplot as plot

In [2]:
sys.path.insert(1,'../')
import preprocess
p1="../../timeseries_csv/ts_wp/"
p2="../../../targets/"

ds47=preprocess._process_dataset(glob.glob(p1+'*2010OMAIS*')[0])
tcwp47=ds47.values
var_names=ds47.columns.values.tolist()

In [3]:
from sklearn.metrics import r2_score,mean_squared_error
from natsort import natsorted
def depickle(loc=None):
    output = []
    with open(loc,'rb') as f:
        output.append(pickle.load(f))
    return output[0]

def flatten(l):
    return [item for sublist in l for item in sublist]

def remove_strange_pred_aligned(TYPE='pmin',pred=None,truth=None):
    truth=np.asarray(truth)
    pred=np.asarray(pred)
    truth = truth[(pred>0) & (pred<10000)]
    pred = pred[(pred>0) & (pred<10000)]
    return truth,pred

def give_r2(filelocs='./WPAC/pcmci_2tau16/causalwpac_pmin_2tau16.obj.',
            suffix=str('*12348'),pc_alpha=[0.1]):
    store_r2train,store_r2test_old,store_r2test_new,store_size = [],[],[],[]
    #for pc_alpha in tqdm([0.01,0.05,0.07,0.09,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]):
    for pc_alpha in tqdm(pc_alpha):
        #filelist = sorted(glob.glob('./combinedscript/pickleddata2/12348/*pmin*.obj.0.2.*.12348'))
        filelistt =sorted(glob.glob(filelocs+str(pc_alpha)+suffix))[5::-1]+\
        sorted(glob.glob(filelocs+str(pc_alpha)+suffix))[6:]
        files = [depickle(obj) for obj in filelistt]
        temp_r2train = [r2_score(file['y']['train'],file['mlr'].predict(file['X']['train'])) for file in files]
        temp_r2test = [r2_score(file['y']['valid']+file['y']['test'],\
                                file['mlr'].predict(file['X']['valid']+file['X']['test'])) for file in files]
        temp = [np.asarray([list(obj) for obj in file['X']['train']]).shape[1] for file in files]
        store_r2train.append(temp_r2train)
        store_r2test.append(temp_r2test)
        store_size.append(temp)
    storedict_pcmci = {'r2_train':store_r2train,'r2_test':store_r2test,'size':store_size}
    del store_r2train,store_r2test,store_size
    return storedict_pcmci

def give_r2_pcstable(filelocs='./WPAC/pcstable_2tau16/newcausalwpac_2-16_pmin*',filelists=None):
    store_r2train,store_r2test_old,store_r2test_new,store_size = [],[],[],[]
    for pc_alpha in tqdm([0.2,0.1,0.05,0.01,1e-3,1e-4,1e-5,1e-6,1e-7,1e-8]):
        filelist = sorted(glob.glob(filelocs))
        files = [depickle(obj) for obj in filelist]
        temp_r2train = [r2_score(file['y']['train'],file['mlr'].predict(file['X']['train'])) for file in files]
        temp_r2test = [r2_score(file['y']['valid']+file['y']['test'],\
                                file['mlr'].predict(file['X']['valid']+file['X']['test'])) for file in files]
        temp = [np.asarray([list(obj) for obj in file['X']['train']]).shape[1] for file in files]
        store_r2train.append(temp_r2train)
        store_r2test.append(temp_r2test)
        store_size.append(temp)
    storedict_pcstable = {'r2_train':store_r2train,'r2_test':store_r2test,'size':store_size}
    del store_r2train,store_r2test,store_size
    return storedict_pcstable

def give_r2_aligned(filelist=None,modelname='mlr'):
    store_r2train,store_r2test_old,store_r2test_new,store_size = [],[],[],[]
    files = [depickle(obj) for obj in filelist]
    for file in tqdm(files):
        temp_r2train = r2_score(file['y']['train'],file[modelname].predict(file['X']['train']))
        temp_r2test_old = r2_score(file['y']['valid']+file['y']['test'],file[modelname].predict(file['X']['valid']+file['X']['test']))
        temp_r2test_new = r2_score(file['y']['newtest'],file[modelname].predict(file['X']['newtest']))
        temp = np.asarray([list(obj) for obj in file['X']['train']]).shape[1]
        store_r2train.append(temp_r2train)
        store_r2test_old.append(temp_r2test_old)
        store_r2test_new.append(temp_r2test_new)
        store_size.append(temp)
    storedict_pcstable = {'r2_train':store_r2train,'r2_test_old':store_r2test_old,'r2_test_new':store_r2test_new,'size':store_size}
    del store_r2train,store_r2test_old,store_r2test_new,store_size
    return storedict_pcstable

from sklearn.metrics import mean_squared_error,mean_absolute_error
def give_rmse_mae_aligned(filelist=None,TYPE='RMSE',modelname='mlr'):
    store_r2train,store_r2test_old,store_r2test_new,store_size = [],[],[],[]
    files = [depickle(obj) for obj in filelist]
    for file in files:
        if TYPE=='RMSE':
            temp_r2train = mean_squared_error(file['y']['train'],file[modelname].predict(file['X']['train']))
            temp_r2test_old = mean_squared_error(file['y']['valid']+file['y']['test'],file[modelname].predict(file['X']['valid']+file['X']['test']))
            temp_r2test_new = mean_squared_error(file['y']['newtest'],file[modelname].predict(file['X']['newtest']))
        elif TYPE=='MAE':
            temp_r2train = mean_absolute_error(file['y']['train'],file[modelname].predict(file['X']['train']))
            temp_r2test_old = mean_absolute_error(file['y']['valid']+file['y']['test'],file[modelname].predict(file['X']['valid']+file['X']['test']))
            temp_r2test_new = mean_absolute_error(file['y']['newtest'],file[modelname].predict(file['X']['newtest']))
        temp = np.asarray([list(obj) for obj in file['X']['train']]).shape[1]
        store_r2train.append(temp_r2train)
        store_r2test_old.append(temp_r2test_old)
        store_r2test_new.append(temp_r2test_new)
        store_size.append(temp)
    storedict_pcstable = {'train':store_r2train,'test_old':store_r2test_old,'test_new':store_r2test_new,'size':store_size}
    del store_r2train,store_r2test_old,store_r2test_new,store_size
    return storedict_pcstable

def give_r2_random(filelocs='./pickleddata_random/12348/randomwpall_2_16*pmin*'):
    random_files = sorted(glob.glob(filelocs))
    store_r2train,store_r2test_old,store_r2test_new,store_size = [],[],[],[]
    for pc_alpha in tqdm(range(len(random_files))):
        #filelist = sorted(glob.glob('./combinedscript/pickleddata2/12348/*pmin*.obj.0.2.*.12348'))
        files = [depickle(obj) for obj in random_files]
        temp_r2train = [r2_score(file['y']['train'],file['mlr'].predict(file['X']['train'])) for file in files]
        temp_r2test_old = [r2_score(file['y']['valid']+file['y']['test'],\
                                file['mlr'].predict(file['X']['valid']+file['X']['test'])) for file in files]
        temp_r2test_new = [r2_score(file['y']['newtest'],file['mlr'].predict(file['X']['newtest'])) for file in files]
        temp = [np.asarray([list(obj) for obj in file['X']['train']]).shape[1] for file in files]
        store_r2train.append(temp_r2train)
        store_r2test_old.append(temp_r2test_old)
        store_r2test_new.append(temp_r2test_new)
        storenatsort_size.append(temp)
    storedict_pcstable = {'r2_train':store_r2train,'r2_test_old':store_r2test_old,'r2_test_new':store_r2test_new,'size':store_size}
    del store_r2train,store_r2test_old,store_r2test_new,store_size
    return storedict_random

def give_r2_lag(filelocs='./pickleddata_lag/12348/lagwpacall8_24*pmin*'):
    fileslist = glob.glob(filelocs)[5::-1]+glob.glob(filelocs)[6:]
    files = [depickle(obj) for obj in fileslist]
    temp_r2train = [r2_score(file['y']['train'],file['mlr'].predict(file['X']['train'])) for file in files]
    temp_r2test_old = [r2_score(file['y']['valid']+file['y']['test'],file['mlr'].predict(file['X']['valid']+file['X']['test'])) for file in files]
    temp_r2test_new = [r2_score(file['y']['newtest'],file['mlr'].predict(file['X']['newtest'])) for file in files]
    temp = [np.asarray([list(obj) for obj in file['X']['train']]).shape[1] for file in files]
    storedict_lag = {'r2_train':temp_r2train,'r2_test_old':temp_r2test_old,'r2_test_new':temp_r2test_new,'size':temp}
    del temp_r2train,temp_r2test_old,temp_r2test_new
    return storedict_lag

def give_r2_anatsortligned_rf(filelist=None,alt_filelist=None):
    store_r2train,store_r2test_old,store_r2test_new,store_size = [],[],[],[]
    files = [depickle(obj) for obj in filelist]
    fileXy = [depickle(obj) for obj in alt_filelist]
    for ind,file in enumerate(files):
        temp_r2train = r2_score(fileXy[ind]['y']['train'],file.predict(fileXy[ind]['X']['train']))
        temp_r2test_old = r2_score(fileXy[ind]['y']['valid']+fileXy[ind]['y']['test'],file.predict(fileXy[ind]['X']['valid']+fileXy[ind]['X']['test']))
        temp_r2test_new = r2_score(fileXy[ind]['y']['newtest'],file.predict(fileXy[ind]['X']['newtest']))
        temp = np.asarray([list(obj) for obj in fileXy[ind]['X']['train']]).shape[1]
        store_r2train.append(temp_r2train)
        store_r2test_old.append(temp_r2test_old)
        store_r2test_new.append(temp_r2test_new)
        store_size.append(temp)
    storedict_pcstable = {'r2_train':store_r2train,'r2_test_old':store_r2test_old,'r2_test_new':store_r2test_new,'size':store_size}
    del store_r2train,store_r2test_old,store_r2test_new,store_size
    return storedict_pcstable

def give_rmse_mae_aligned_rf(filelist=None,TYPE='RMSE',alt_filelist=None):
    store_r2train,store_r2test_old,store_r2test_new,store_size = [],[],[],[]
    files = [depickle(obj) for obj in filelist]
    fileXy = [depickle(obj) for obj in alt_filelist]
    for ind,file in tqdm(enumerate(files)):
        if TYPE=='RMSE':
            temp_r2train = mean_squared_error(fileXy[ind]['y']['train'],file.predict(fileXy[ind]['X']['train']))
            temp_r2test_old = mean_squared_error(fileXy[ind]['y']['valid']+fileXy[ind]['y']['test'],file.predict(fileXy[ind]['X']['valid']+fileXy[ind]['X']['test']))
            temp_r2test_new = mean_squared_error(fileXy[ind]['y']['newtest'],file.predict(fileXy[ind]['X']['newtest']))
        elif TYPE=='MAE':
            temp_r2train = mean_absolute_error(fileXy[ind]['y']['train'],file.predict(fileXy[ind]['X']['train']))
            temp_r2test_old = mean_absolute_error(fileXy[ind]['y']['valid']+fileXy[ind]['y']['test'],file.predict(fileXy[ind]['X']['valid']+fileXy[ind]['X']['test']))
            temp_r2test_new = mean_absolute_error(fileXy[ind]['y']['newtest'],file.predict(fileXy[ind]['X']['newtest']))
        temp = np.asarray([list(obj) for obj in fileXy[ind]['X']['train']]).shape[1]
        store_r2train.append(temp_r2train)
        store_r2test_old.append(temp_r2test_old)
        store_r2test_new.append(temp_r2test_new)
        store_size.append(temp)
    storedict_pcstable = {'train':store_r2train,'test_old':store_r2test_old,'test_new':store_r2test_new,'size':store_size}
    del store_r2train,store_r2test_old,store_r2test_new,store_size
    return storedict_pcstable

def give_rmse_mae_aligned_rf2(filelist=None,TYPE='RMSE',alt_filelist=None):
    store_r2train,store_r2test_old,store_r2test_new,store_size = [],[],[],[]
    files = [depickle(obj) for obj in filelist]
    fileXy = [depickle(obj) for obj in alt_filelist]
    for ind,file in tqdm(enumerate(files)):
        if TYPE=='RMSE':
            temp_r2train = mean_squared_error(fileXy[ind]['y']['train'],file['model'].predict(fileXy[ind]['X']['train']))
            temp_r2test_old = mean_squared_error(fileXy[ind]['y']['valid']+fileXy[ind]['y']['test'],file['model'].predict(fileXy[ind]['X']['valid']+fileXy[ind]['X']['test']))
            temp_r2test_new = mean_squared_error(fileXy[ind]['y']['newtest'],file['model'].predict(fileXy[ind]['X']['newtest']))
        elif TYPE=='MAE':
            temp_r2train = mean_absolute_error(fileXy[ind]['y']['train'],file['model'].predict(fileXy[ind]['X']['train']))
            temp_r2test_old = mean_absolute_error(fileXy[ind]['y']['valid']+fileXy[ind]['y']['test'],file['model'].predict(fileXy[ind]['X']['valid']+fileXy[ind]['X']['test']))
            temp_r2test_new = mean_absolute_error(fileXy[ind]['y']['newtest'],file['model'].predict(fileXy[ind]['X']['newtest']))
        temp = np.asarray([list(obj) for obj in fileXy[ind]['X']['train']]).shape[1]
        store_r2train.append(temp_r2train)
        store_r2test_old.append(temp_r2test_old)
        store_r2test_new.append(temp_r2test_new)
        store_size.append(temp)
    storedict_pcstable = {'train':store_r2train,'test_old':store_r2test_old,'test_new':store_r2test_new,'size':store_size}
    del store_r2train,store_r2test_old,store_r2test_new,store_size
    return storedict_pcstable

def give_r2_aligned_rf(filelist=None,alt_filelist=None):
    store_r2train,store_r2test_old,store_r2test_new,store_size = [],[],[],[]
    files = [depickle(obj) for obj in filelist]
    fileXy = [depickle(obj) for obj in alt_filelist]
    for ind,file in tqdm(enumerate(files)):
        temp_r2train = r2_score(fileXy[ind]['y']['train'],file.predict(fileXy[ind]['X']['train']))
        temp_r2test_old = r2_score(fileXy[ind]['y']['valid']+fileXy[ind]['y']['test'],file.predict(fileXy[ind]['X']['valid']+fileXy[ind]['X']['test']))
        temp_r2test_new = r2_score(fileXy[ind]['y']['newtest'],file.predict(fileXy[ind]['X']['newtest']))
        temp = np.asarray([list(obj) for obj in fileXy[ind]['X']['train']]).shape[1]
        store_r2train.append(temp_r2train)
        store_r2test_old.append(temp_r2test_old)
        store_r2test_new.append(temp_r2test_new)
        store_size.append(temp)
    storedict_pcstable = {'r2_train':store_r2train,'r2_test_old':store_r2test_old,'r2_test_new':store_r2test_new,'size':store_size}
    del store_r2train,store_r2test_old,store_r2test_new,store_size
    return storedict_pcstable

def give_r2_aligned_rf2(filelist=None,alt_filelist=None):
    store_r2train,store_r2test_old,store_r2test_new,store_size = [],[],[],[]
    files = [depickle(obj) for obj in filelist]
    fileXy = [depickle(obj) for obj in alt_filelist]
    for ind,file in tqdm(enumerate(files)):
        temp_r2train = r2_score(fileXy[ind]['y']['train'],file['model'].predict(fileXy[ind]['X']['train']))
        temp_r2test_old = r2_score(fileXy[ind]['y']['valid']+fileXy[ind]['y']['test'],file['model'].predict(fileXy[ind]['X']['valid']+fileXy[ind]['X']['test']))
        temp_r2test_new = r2_score(fileXy[ind]['y']['newtest'],file['model'].predict(fileXy[ind]['X']['newtest']))
        temp = np.asarray([list(obj) for obj in fileXy[ind]['X']['train']]).shape[1]
        store_r2train.append(temp_r2train)
        store_r2test_old.append(temp_r2test_old)
        store_r2test_new.append(temp_r2test_new)
        store_size.append(temp)
    storedict_pcstable = {'r2_train':store_r2train,'r2_test_old':store_r2test_old,'r2_test_new':store_r2test_new,'size':store_size}
    del store_r2train,store_r2test_old,store_r2test_new,store_size
    return storedict_pcstable

In [4]:
from natsort import natsorted

In [5]:
#####################################################################################################################################
# Read files (PC1)
listtt = (sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/parallel_tigramite/newdata/pcstablewpac*aligned_8-24*v10*')))
# Wrong order to correct...
aligned2_16_list = [listtt[-1],listtt[-4],listtt[-2],listtt[-5],listtt[-3],listtt[-6]]
for obj in listtt[:-6]:
    aligned2_16_list.append(obj)
#####################################################################################################################################    
#pcmci_wpac_2tau16 = give_r2()
pcstable_wpac_2tau16 = give_r2_aligned(aligned2_16_list)

  0%|          | 0/39 [00:00<?, ?it/s]

In [6]:
rmse_wpac_2tau16 = give_rmse_mae_aligned(aligned2_16_list,'RMSE')
mae_wpac_2tau16 = give_rmse_mae_aligned(aligned2_16_list,'MAE')

In [7]:
r2_df = pd.DataFrame({'train':np.asarray(pcstable_wpac_2tau16['r2_train']).flatten(),'test':np.asarray(pcstable_wpac_2tau16['r2_test']).flatten(),'name':aligned2_16_list})
rmse_df = pd.DataFrame({'train':np.asarray(rmse_wpac_2tau16['train']).flatten(),'test':np.asarray(rmse_wpac_2tau16['test']).flatten(),'name':aligned2_16_list})
mae_df = pd.DataFrame({'train':np.asarray(mae_wpac_2tau16['train']).flatten(),'test':np.asarray(mae_wpac_2tau16['test']).flatten(),'name':aligned2_16_list})

KeyError: 'r2_test'

In [ ]:
maxr2 = np.asarray(pcstable_wpac_2tau16['r2_test']).max()
index_r2_best,index_r2_1percent = r2_df['test'].idxmax(),r2_df[r2_df['test']>maxr2-maxr2*0.01].index
index_r2_best
#import rfreg_funcs
#rfreg_funcs.save_to_pickle('./tmp/multimodelnames_fig4.pkl',list(r2_df.iloc[index_r2_1percent]['name'].values))

In [ ]:
import rfreg_funcs
lag_wpac_2tau16 = give_r2_lag('./pickleddata_lag/12348/lagwpacall8_24*v10*')
storelag = {'filename':glob.glob('./pickleddata_lag/12348/lagwpacall8_24*v10*')[5::-1]+glob.glob('./pickleddata_lag/12348/lagwpacall8_24*v10*')[6:],
            'r2_test': lag_wpac_2tau16['r2_test'],'r2_train':lag_wpac_2tau16['r2_train']}
print(index_r2_best)
#rfreg_funcs.save_to_pickle('./tmp/multimodelnames_fig4_lag.pkl',list(df_lag.iloc[index_r2_1percent]['filename'].values))

#### Random Feature Selection MLR

In [7]:
def output_performance_r2(listt=None,var='R2',TYPE='MLR',listtt_mlr=None,modelname='mlr'):
    if var=='R2':
        if TYPE=='MLR':
            files = give_r2_aligned(listt,modelname)
        elif TYPE=='RF':
            from natsort import natsorted
            #####################################################################################################################################
            # Read files (PC1)
            #listtt = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/parallel_tigramite/newdata/rf/*pmin*'))
            # Wrong order to correct...
            aligned2_16_list = [listt[-1],listt[-4],listt[-2],listt[-5],listt[-3],listt[-6]]
            for obj in listt[:-6]:
                aligned2_16_list.append(obj)
            #####################################################################################################################################
            # Read files (PC1)
            #listtt_mlr = (sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/parallel_tigramite/newdata/pcstablewpac_aligned_8-24_pmin*')))
            # Wrong order to correct...
            aligned2_16_list_mlr = [listtt_mlr[-1],listtt_mlr[-4],listtt_mlr[-2],listtt_mlr[-5],listtt_mlr[-3],listtt_mlr[-6]]
            for obj in listtt_mlr[:-6]:
                aligned2_16_list_mlr.append(obj)
            #####################################################################################################################################
            files = give_r2_aligned_rf(aligned2_16_list,aligned2_16_list_mlr)
            #try:
            #    print('hi')
            #    files = give_r2_aligned_rf(aligned2_16_list,aligned2_16_list_mlr)
            #except:
            #    print('hihi')
            #    files = give_r2_aligned_rf2(aligned2_16_list,aligned2_16_list_mlr)

        storelag = {'filename':listt,'r2_test_new': files['r2_test_new'],'r2_train':files['r2_train'],'r2_valid':files['r2_test_old']}
        r2_df = pd.DataFrame({'train':np.asarray(storelag['r2_train']).flatten(),\
                              'test':np.asarray(storelag['r2_test_new']).flatten(),\
                              'valid':np.asarray(storelag['r2_valid']).flatten(),\
                              'name':storelag['filename']})
        df_lag = pd.DataFrame(storelag)
        maxr2 = np.asarray(df_lag['r2_valid']).max()
        index_r2_best,index_r2_1percent = df_lag['r2_valid'].idxmax(),df_lag[df_lag['r2_valid']>maxr2-maxr2*0.10].index
        return storelag,float(r2_df.iloc[index_r2_best]['train']), float(r2_df.iloc[index_r2_best]['valid']), float(r2_df.iloc[index_r2_best]['test']),\
    float(np.asarray(files['size'])[index_r2_best])

def output_performance_mae_rmse(listt=None,var='RMSE',TYPE='MLR',listtt_mlr=None,modelname='mlr',r2dict=None):
    if var=='MAE':
        if TYPE=='MLR':
            files = give_rmse_mae_aligned(listt,var,modelname)
        elif TYPE=='RF':
            from natsort import natsorted
            #####################################################################################################################################
            # Read files (PC1)
            #listtt = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/parallel_tigramite/newdata/rf/*pmin*'))
            # Wrong order to correct...
            aligned2_16_list = [listt[-1],listt[-4],listt[-2],listt[-5],listt[-3],listt[-6]]
            for obj in listt[:-6]:
                aligned2_16_list.append(obj)
            #####################################################################################################################################
            # Read files (PC1)
            #listtt_mlr = (sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/parallel_tigramite/newdata/pcstablewpac_aligned_8-24_pmin*')))
            # Wrong order to correct...
            aligned2_16_list_mlr = [listtt_mlr[-1],listtt_mlr[-4],listtt_mlr[-2],listtt_mlr[-5],listtt_mlr[-3],listtt_mlr[-6]]
            for obj in listtt_mlr[:-6]:
                aligned2_16_list_mlr.append(obj)
            #####################################################################################################################################
            files = give_rmse_mae_aligned_rf(aligned2_16_list,var,aligned2_16_list_mlr)
            #try:
            #    files = give_rmse_mae_aligned_rf(aligned2_16_list,var,aligned2_16_list_mlr)
            #except:
            #    files = give_rmse_mae_aligned_rf2(aligned2_16_list,var,aligned2_16_list_mlr)
                
        r2_df = pd.DataFrame({'r2_train':np.asarray(r2dict['r2_train']).flatten(),'r2_valid':np.asarray(r2dict['r2_valid']).flatten(),\
                              'r2_test':np.asarray(r2dict['r2_test_new']).flatten(),'name':listt})
        mae_df = pd.DataFrame({'train':np.asarray(files['train']).flatten(),'valid':np.asarray(files['test_old']).flatten(),\
                               'test':np.asarray(files['test_new']).flatten(),'name':listt})
        df_lag = pd.DataFrame(r2_df)
        maxr2 = np.asarray(df_lag['r2_valid']).max()
        index_r2_best = df_lag['r2_valid'].idxmax()
        return float(mae_df.iloc[index_r2_best]['train']),float(mae_df.iloc[index_r2_best]['valid']),\
    float(mae_df.iloc[index_r2_best]['test']),float(np.asarray(files['size'])[index_r2_best])
    
    elif var=='RMSE':
        if TYPE=='MLR':
            files = give_rmse_mae_aligned(listt,var,modelname)
        elif TYPE=='RF':
            from natsort import natsorted
            #####################################################################################################################################
            # Read files (PC1)
            #listtt = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/parallel_tigramite/newdata/rf/*pmin*'))
            # Wrong order to correct...
            aligned2_16_list = [listt[-1],listt[-4],listt[-2],listt[-5],listt[-3],listt[-6]]
            for obj in listt[:-6]:
                aligned2_16_list.append(obj)
            #####################################################################################################################################
            # Read files (PC1)
            #listtt_mlr = (sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/parallel_tigramite/newdata/pcstablewpac_aligned_8-24_pmin*')))
            # Wrong order to correct...
            aligned2_16_list_mlr = [listtt_mlr[-1],listtt_mlr[-4],listtt_mlr[-2],listtt_mlr[-5],listtt_mlr[-3],listtt_mlr[-6]]
            for obj in listtt_mlr[:-6]:
                aligned2_16_list_mlr.append(obj)
            #####################################################################################################################################
            files = give_rmse_mae_aligned_rf(aligned2_16_list,var,aligned2_16_list_mlr)
        r2_df = pd.DataFrame({'r2_train':np.asarray(r2dict['r2_train']).flatten(),'r2_valid':np.asarray(r2dict['r2_valid']).flatten(),\
                              'r2_test':np.asarray(r2dict['r2_test_new']).flatten(),'name':listt})
        rmse_df = pd.DataFrame({'train':np.asarray(files['train']).flatten(),'valid':np.asarray(files['test_old']).flatten(),\
                               'test':np.asarray(files['test_new']).flatten(),'name':listt})
        df_lag = pd.DataFrame(r2_df)
        maxr2 = np.asarray(df_lag['r2_valid']).max()
        index_r2_best = df_lag['r2_valid'].idxmax()
        return float(rmse_df.iloc[index_r2_best]['train']),float(rmse_df.iloc[index_r2_best]['valid']),\
    float(rmse_df.iloc[index_r2_best]['test']),float(np.asarray(files['size'])[index_r2_best])
    
def storeRMSE(path_prefix=None,TYPE='MLR',storename='mlr',alt_fileprefix=None,natsortted='No',modelstart=0):
    storeRMSE = []
    for target in ['pmin','v10','precip']:
        #filelist = glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/lag/12348/lagwpac*8_24*'+str(target)+'.obj.*')
        
        if natsortted=='Yes':
            listtt_lag = natsorted(glob.glob(path_prefix+str(target)+'.obj.*'))[int(modelstart):]
        elif natsortted=='No':
            listtt_lag = sorted(glob.glob(path_prefix+str(target)+'.obj.*'))[int(modelstart):]
            
        if TYPE=='MLR':
            storer2,_,_,_,_ = output_performance_r2(listtt_lag,'R2',TYPE,None,storename)
            train,valid,test,_ = output_performance_mae_rmse(listtt_lag,'RMSE',TYPE,None,storename,storer2)
        elif TYPE=='RF':
            if natsortted=='Yes':
                alt_filelist = natsorted(glob.glob(alt_fileprefix+str(target)+'*'))[int(modelstart):]
            elif natsortted=='No':
                alt_filelist = sorted(glob.glob(alt_fileprefix+str(target)+'*'))[int(modelstart):]
            storer2,_,_,_,_ = output_performance_r2(listtt_lag,'R2',TYPE,alt_filelist,storename)
            train,valid,test,_ = output_performance_mae_rmse(listtt_lag,'RMSE',TYPE,alt_filelist,storename,storer2)    
        
        if target=='precip':
            storeRMSE.append([train/1000,valid/1000,test/1000])
        else:
            storeRMSE.append([train,valid,test])
    return [np.round(obj,decimals=2) for obj in np.ravel(np.asarray(storeRMSE),'F')]

def storeMAE(path_prefix=None,TYPE='MLR',storename='mlr',alt_fileprefix=None,natsortted='No',modelstart=0):
    storeMAE = []
    for target in ['pmin','v10','precip']:
        #filelist = glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/lag/12348/lagwpac*8_24*'+str(target)+'.obj.*')
        if natsortted=='Yes':
            listtt_lag = natsorted(glob.glob(path_prefix+str(target)+'.obj.*'))[int(modelstart):]
        elif natsortted=='No':
            listtt_lag = sorted(glob.glob(path_prefix+str(target)+'.obj.*'))[int(modelstart):]
            
        if TYPE=='MLR':
            storer2,_,_,_,_ = output_performance_r2(listtt_lag,'R2',TYPE,None,storename)
            train,valid,test,_ = output_performance_mae_rmse(listtt_lag,'MAE',TYPE,None,storename,storer2)
        elif TYPE=='RF':
            if natsortted=='Yes':
                alt_filelist = natsorted(glob.glob(alt_fileprefix+str(target)+'*'))[int(modelstart):]
            elif natsortted=='No':
                alt_filelist = sorted(glob.glob(alt_fileprefix+str(target)+'*'))[int(modelstart):]
            storer2,_,_,_,_ = output_performance_r2(listtt_lag,'R2',TYPE,alt_filelist,storename)
            train,valid,test,_ = output_performance_mae_rmse(listtt_lag,'MAE',TYPE,alt_filelist,storename,storer2)   
        if target=='precip':
            storeMAE.append([train/1000,valid/1000,test/1000])
        else:
            storeMAE.append([train,valid,test])
    return [np.round(obj,decimals=2) for obj in np.ravel(np.asarray(storeMAE),'F')]

In [8]:
# random (start from 9)
listtt = (natsorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/random_rf/randomrf_8-24*v10.obj.*')))[9:]
listtt_mlr = (natsorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/random/12348/randomwp*8_24_v10*')))[9:]
storer2,train_r2,valid_r2,test_r2,size = output_performance_r2(listtt,'R2','RF',listtt_mlr,'model')

0it [00:00, ?it/s]

In [9]:
train_r2,valid_r2,test_r2,size

(0.9292547666305121, 0.7679355368775268, 0.765194226976501, 770.0)

## Creating Array for MAE - noncausal MLR

In [10]:
All=/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/noncausal/12348/*precip*
lagged=/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/lag/12348/lagwpac*8_24*precip.obj.*
random=/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/random/12348/random*8_24_precip.obj.*
xai=/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/rf_feature/12348/rfredall8_24_precip.obj.*

SyntaxError: invalid syntax (1501488782.py, line 1)

In [10]:
allmodl = []
for target in ['pmin','v10','precip']:
    model = depickle(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/noncausal/12348/*'+str(target)+'*')[0])
    if target=='precip':
        allmodl.append([mean_squared_error(model['y']['train'],model['mlr'].predict(model['X']['train']))/1000,\
                        mean_squared_error(model['y']['valid']+model['y']['test'],model['mlr'].predict(model['X']['valid']+model['X']['test']))/1000,\
                        mean_squared_error(model['y']['newtest'],model['mlr'].predict(model['X']['newtest']))/1000])
    else:
        allmodl.append([mean_squared_error(model['y']['train'],model['mlr'].predict(model['X']['train'])),\
                        mean_squared_error(model['y']['valid']+model['y']['test'],model['mlr'].predict(model['X']['valid']+model['X']['test'])),\
                        mean_squared_error(model['y']['newtest'],model['mlr'].predict(model['X']['newtest']))])

In [13]:
laglist = storeMAE('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/lag/12348/lagwpac*8_24*')
randomlist = storeMAE('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/random/12348/random*8_24*')
xailist = storeMAE('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/rf_feature/12348/rfregall8_24*')

  0%|          | 0/107 [00:00<?, ?it/s]

  0%|          | 0/107 [00:00<?, ?it/s]

  0%|          | 0/107 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

In [16]:
mlr_fileprefix = '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/parallel_tigramite/newdata/pcstablewpac*8-24_*'
causal_rf = storeMAE('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/parallel_tigramite/newdata/rf/rfmodels_8-24_*','RF','model',mlr_fileprefix,'No',0)
causal_mlr = storeMAE('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/parallel_tigramite/newdata/pcstablewpac*8-24_*','MLR','mlr',None,'Yes',0)
#lstm = storeRMSE('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/best_lstm/best_model3*')

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

In [26]:

rfreg_funcs.save_to_pickle('./store/rmse_mae/storestuff_causalmlr_mae.pkl',causal_mlr)

In [11]:
import rfreg_funcs,torch
import torch
from torch import nn
class MyLSTM(nn.Module):
    def __init__(self,
                 input_size,
                hidden_units,
                dropout_rates):
        super(MyLSTM, self).__init__()
        self.units = hidden_units
        self.drates = dropout_rates
        
        self.lstm_layer = nn.LSTM(input_size = input_size,
                                 hidden_size = self.units,
                                 num_layers = 1,
                                 bias = True,
                                 batch_first = True)
        
        self.dropout_layer = nn.Dropout(p = self.drates)
        
        self.dense_layer = nn.Linear(in_features = self.units, 
                                     out_features = self.units)
        
        self.output_layer = nn.Linear(in_features = self.units,
                                      out_features=1)
        
    def forward(self, X):
        output, (h_n, c_n) = self.lstm_layer(X)
        
        hidden_state = self.dropout_layer(h_n[0])
        #print(f'1st Hidden {hidden_state.shape}')
        hidden_state2 = torch.tanh(self.dense_layer(hidden_state))
        #print(f'2nd Hidden {hidden_state2.shape}')
        p_hat = torch.flatten(self.output_layer((hidden_state2)))
        
        return p_hat

In [12]:
torch.load(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/best_lstm/best_model3*')[0])

MyLSTM(
  (lstm_layer): LSTM(234, 69, batch_first=True)
  (dropout_layer): Dropout(p=0, inplace=False)
  (dense_layer): Linear(in_features=69, out_features=69, bias=True)
  (output_layer): Linear(in_features=69, out_features=1, bias=True)
)

In [ ]:
noncausal_rf='/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/rf_feature/12348/rfregall8_24*'
rf_lagged='/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/lag_rf/lagrf_8-24*'
rf_random='/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/random_rf/randomrf_8-24*'

In [19]:
# Lag uses 'MLR' because we saved the X,y
mlr_fileprefix = '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/lag/12348/lagwpac*8_24*'
rf_lagged = storeMAE('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/lag_rf/lagrf_8-24*',
                      'MLR','model',mlr_fileprefix,'Yes',0)
print(rf_lagged)

  0%|          | 0/107 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

  0%|          | 0/107 [00:00<?, ?it/s]

[2.04, 1.12, 0.14, 3.78, 1.95, 0.22, 3.41, 1.85, 0.23]


In [20]:
mlr_fileprefix = '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/random/12348/randomwp*8_24*'
rf_random = storeMAE('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/random_rf/randomrf_8-24*',
                      'RF','model',mlr_fileprefix,'Yes',9)
print(rf_random)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[2.07, 1.12, 0.12, 4.43, 2.2, 0.25, 3.94, 2.1, 0.25]


In [140]:
def handle_lines(prefix=None,numbers=None):
    for obj in [np.round(obj, decimals=2) for obj in list(numbers)]:
        tofill = str(obj)+" & "
        prefix+=tofill
    return prefix

In [138]:
alllist = [np.round(obj,decimals=2) for obj in np.ravel(np.asarray(allmodl),'F')]
storestuff = np.vstack([np.asarray(obj) for obj in [alllist,laglist,randomlist,xailist]])

In [278]:
import rfreg_funcs
rfreg_funcs.save_to_pickle('./store/rmse_mae/storestuff_noncausalrf_lagged_rmse.pkl',rf_lagged)
rfreg_funcs.save_to_pickle('./store/rmse_mae/storestuff_noncausalrf_random_rmse.pkl',rf_random)

In [144]:
path = './testtable_rmse_noncausal_mlr.txt'
f= open(path,"w+")
f.write("\\begin{table}\n")
f.write("\\begin{centering}\n")
f.write("\\begin{tabular}{cc|c|c|c|c|c|c|c|c|c|c}\n")
f.write("\\multicolumn{2}{c}{ML Models} & \multicolumn{3}{c}{Training} & \multicolumn{3}{c}{Validation} & \multicolumn{3}{c|}{Test } & \\tabularnewline")
f.write("\\hline\n")

sentence1 = "\\multicolumn{2}{c|}{\\textbf{Target}} & Pmin (hPa)\\textcolor{purple}{{} } & V10 (m/s) & Precip (km\\textasciicircum 3) & Pmin (hPa)\\textcolor{purple}{{} } & V10 (m/s) & "+\
"Precip (km\\textasciicircum 3) & Pmin (hPa)\\textcolor{purple}{{} } & V10 (m/s) & Precip (km\\textasciicircum 3) & \\tabularnewline "+\
"\cline{3-12} \cline{4-12} \cline{5-12} \cline{6-12} \cline{7-12} \cline{8-12} \cline{9-12} \cline{10-12} \cline{11-12} \cline{12-12}\n"
f.write(sentence1)
f.write(handle_lines('\\multicolumn{2}{c|}{\\textbf{Causal RF}} & ' ,storestuff[0,:])+"\\tabularnewline\n")
f.write(handle_lines('\\multirow{4}{*}{\\textbf{Non-causal MLR}} & All & ',storestuff[0,:])+"\\tabularnewline\n")
f.write(handle_lines('\cline{2-12} \cline{3-12} \cline{4-12} \cline{5-12} \cline{6-12} \cline{7-12} \cline{8-12} \cline{9-12} \cline{10-12} \cline{11-12} \cline{12-12} & Lagged & ',
                     storestuff[1,:])+"\\tabularnewline\n")
f.write(handle_lines('\cline{2-12} \cline{3-12} \cline{4-12} \cline{5-12} \cline{6-12} \cline{7-12} \cline{8-12} \cline{9-12} \cline{10-12} \cline{11-12} \cline{12-12} & Random & ',
                     storestuff[2,:])+"\\tabularnewline\n")
f.write(handle_lines('\cline{2-12} \cline{3-12} \cline{4-12} \cline{5-12} \cline{6-12} \cline{7-12} \cline{8-12} \cline{9-12} \cline{10-12} \cline{11-12} \cline{12-12} & XAI & ',
                     storestuff[3,:])+"\\tabularnewline\n")
f.write("\\hline\n")
f.write("\\end{tabular}\n")
f.write("\\par\\end{centering}\n")
f.write("\\end{table}")
f.close()
print('Table printed to',path)

Table printed to ./testtable.txt


In [153]:
laglistMAE = storeMAE('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/lag/12348/lagwpac*8_24*')
randomlistMAE = storeMAE('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/random/12348/random*8_24*')
xailistMAE = storeMAE('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/rf_feature/12348/rfregall8_24*')

  0%|          | 0/109 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

In [154]:
allmodlMAE = []
for target in ['pmin','v10','precip']:
    model = depickle(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/causal_notebooks/climateinformatics_revision1/store/noncausal/12348/*'+str(target)+'*')[0])
    if target=='precip':
        allmodlMAE.append([mean_absolute_error(model['y']['train'],model['mlr'].predict(model['X']['train']))/1000,\
                        mean_absolute_error(model['y']['valid']+model['y']['test'],model['mlr'].predict(model['X']['valid']+model['X']['test']))/1000,\
                        mean_absolute_error(model['y']['newtest'],model['mlr'].predict(model['X']['newtest']))/1000])
    else:
        allmodlMAE.append([mean_absolute_error(model['y']['train'],model['mlr'].predict(model['X']['train'])),\
                        mean_absolute_error(model['y']['valid']+model['y']['test'],model['mlr'].predict(model['X']['valid']+model['X']['test'])),\
                        mean_absolute_error(model['y']['newtest'],model['mlr'].predict(model['X']['newtest']))])

In [155]:
alllistMAE = [np.round(obj,decimals=2) for obj in np.ravel(np.asarray(allmodlMAE),'F')]
storestuffMAE = np.vstack([np.asarray(obj) for obj in [alllistMAE,laglistMAE,randomlistMAE,xailistMAE]])

In [157]:
rfreg_funcs.save_to_pickle('./store/rmse_mae/storestuff_noncausalmlr_mae.pkl',storestuffMAE)